TODO:


Need to make sure the classes are actually working the way we want them to, especially in terms of OOP.

Update your GitHub repository with the latest code changes from your local project. Start by staging your modifications and creating a commit. Then, push the changes to your GitHub repository.

Additionally, document your progress by adding to your GitHub README file. You can refer to the relevant lesson in the prerequisites for this task for more information.

At minimum, your README file should contain the following information:

Project Title
Table of Contents, if the README file is long
A description of the project: what it does, the aim of the project, and what you learned
Installation instructions
Usage instructions
File structure of the project
License information
You don't have to write all of this at once, but make sure to update your README file as you go along, so that you don't forget to add anything.

In [1]:
import sys
sys.path.append('../classes/')
from data_cleaning import DataCleaning
from data_extraction import DataExtraction
from database_utils import DatabaseConnector


dc = DatabaseConnector()
de = DataExtraction()
clean = DataCleaning()

# Use your list_db_tables method to get the name of the table containing user data.
table_list = dc.list_db_tables()

# Use the read_rds_table method to extract the table containing user data and return a pandas DataFrame.
df = de.read_rds_table(dc.init_db_engine(), table_list[1])
# print(df.head(10))
clean.clean_user_data(df)


Cleaning mode begun: 
1-- Checking for null or na values
index            0
first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64


TypeError: '>' not supported between instances of 'str' and 'int'

In [4]:
new_index = [x for x in range (0, 20)]
new_index

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [13]:
import pandas as pd

a = {'index': 450, 'address': 'Gotthilf-Rose-Straße 7/3\n45457 Feuchtwangen, Bad Honnef', 'longitude': '50.64336', 'lat': None, 'locality': 'Bad Honnef', 'store_code': 'BA-B4AED588', 'staff_numbers': '36', 'opening_date': '2001-05-12', 'store_type': 'Local', 'latitude': '7.2278', 'country_code': 'DE', 'continent': 'eeEurope'}
b = {'index': 0, 'address': 'N/A', 'longitude': 'N/A', 'lat': 'N/A', 'locality': 'N/A', 'store_code': 'WEB-1388012W', 'staff_numbers': '325', 'opening_date': '2010-06-12', 'store_type': 'Web Portal', 'latitude': None, 'country_code': 'GB', 'continent': 'Europe'}

# df = pd.DataFrame.from_dict(a, orient="index", columns=['address', 'longitude', 'lat', 'locality', 'store_code', 'staff_numbers', 'opening_date', 'store_type', 'latitude', 'country_code', 'continent'])


print(index)
a

# create original dataframe
index = a.pop('index')
df = pd.DataFrame(a, index=[index])

# add to dataframe
index = b.pop('index')
df = pd.concat( [df, pd.DataFrame(b, index=[index]) ], ignore_index=True )

df



0


,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,"Gotthilf-Rose-Straße 7/3\n45457 Feuchtwangen, ...",50.64336,None,Bad Honnef,BA-B4AED588,36,2001-05-12,Local,7.2278,DE,eeEurope
1,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe


In [14]:
for a in range(0,30):
    print(a)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [16]:
a = '10kg'
a.split('kg')[0]

'10'

In [17]:
len('cfe701da-6493-4a7f-83cf-9aab1650f689')

36

In [ ]:
-- assign types for the orders_table
ALTER TABLE orders_table
ALTER COLUMN date_uuid TYPE UUID USING date_uuid::UUID,
    ALTER COLUMN user_uuid TYPE UUID USING user_uuid::UUID,
    ALTER COLUMN card_number TYPE VARCHAR(19),
    ALTER COLUMN store_code TYPE VARCHAR(12),
    ALTER COLUMN product_code TYPE VARCHAR(11),
    ALTER COLUMN product_quantity TYPE SMALLINT;
-- assign types for dim_users
ALTER TABLE dim_users
ALTER COLUMN first_name TYPE VARCHAR(255),
    ALTER COLUMN last_name TYPE VARCHAR(255),
    ALTER COLUMN date_of_birth TYPE DATE,
    ALTER COLUMN country_code TYPE VARCHAR(2),
    ALTER COLUMN user_uuid TYPE UUID USING user_uuid::UUID,
    ALTER COLUMN join_date TYPE DATE;
-- assign types for dim_store_details
ALTER TABLE dim_stores_details
ALTER COLUMN longitude TYPE FLOAT USING longitude::FLOAT,
    ALTER COLUMN locality TYPE VARCHAR(255),
    ALTER COLUMN store_code TYPE VARCHAR(12),
    ALTER COLUMN staff_numbers TYPE SMALLINT,
    ALTER COLUMN opening_date TYPE DATE USING opening_date::DATE,
    ALTER COLUMN store_type TYPE VARCHAR(255),
    ALTER COLUMN store_type DROP NOT NULL,
    ALTER COLUMN latitude TYPE FLOAT USING latitude::FLOAT,
    ALTER COLUMN country_code TYPE VARCHAR(2),
    ALTER COLUMN continent TYPE VARCHAR(255);
-- change NULL values to N/A    
UPDATE dim_stores_details
SET address = 'N/A',
    locality = 'N/A',
    country_code = 'N/A'
WHERE store_code ILIKE 'WEB-1388012W';
-- create weight_class
ALTER TABLE dim_products
ADD weight_class VARCHAR(14);
UPDATE dim_products
SET weight_class = CASE
        WHEN weight < 2 THEN 'Light'
        WHEN weight >= 2
        AND weight < 40 THEN 'Mid_Sized'
        WHEN weight >= 40
        AND weight < 140 THEN 'Heavy'
        WHEN weight > 120 THEN 'Long'
    END;
-- do preparation to make column boolean, then change name to still_available
UPDATE dim_products
SET removed = REPLACE(removed, 'Still_avaliable', 'True');
UPDATE dim_products
SET removed = REPLACE(removed, 'Removed', 'False');
ALTER TABLE dim_products
    RENAME COLUMN "removed" TO still_available;
-- assign types for dim_products 
ALTER TABLE dim_products
ALTER COLUMN product_price TYPE FLOAT,
    ALTER COLUMN weight TYPE FLOAT,
    ALTER COLUMN "EAN" TYPE VARCHAR(17),
    ALTER COLUMN product_code TYPE VARCHAR(11),
    ALTER COLUMN date_added TYPE DATE USING date_added::DATE,
    ALTER COLUMN uuid TYPE UUID USING uuid::UUID,
    ALTER COLUMN still_available TYPE BOOL USING still_available::BOOL,
    ALTER COLUMN weight_class TYPE VARCHAR(9);
-- assign one type for date
ALTER TABLE dim_date_times
ALTER COLUMN date_uuid TYPE UUID USING date_uuid::UUID;
-- assign types for dim_card_details
ALTER TABLE dim_card_details
ALTER COLUMN card_number TYPE VARCHAR(19),
    ALTER COLUMN expiry_date TYPE DATE USING expiry_date::DATE,
    ALTER COLUMN date_payment_confirmed TYPE DATE USING date_payment_confirmed::DATE;
-- set primary keys for all the tables that are referenced in orders_table
ALTER TABLE dim_date_times
ADD PRIMARY KEY (date_uuid);
ALTER TABLE dim_users
ADD PRIMARY KEY (user_uuid);
ALTER TABLE dim_card_details
ADD PRIMARY KEY (card_number);
ALTER TABLE dim_stores_details
ADD PRIMARY KEY (store_code);
ALTER TABLE dim_products
ADD PRIMARY KEY (product_code);
-- set foreign keys orders table to reference our newly created primary keys
ALTER TABLE orders_table
ADD CONSTRAINT fk_orders_table_dim_date_times FOREIGN KEY (date_uuid) REFERENCES dim_date_times(date_uuid);
ALTER TABLE orders_table
ADD CONSTRAINT fk_orders_table_dim_users FOREIGN KEY (user_uuid) REFERENCES dim_users(user_uuid);
ALTER TABLE orders_table
ADD CONSTRAINT fk_orders_table_dim_card_details FOREIGN KEY (card_number) REFERENCES dim_card_details(card_number);
ALTER TABLE orders_table
ADD CONSTRAINT fk_orders_table_dim_stores_details FOREIGN KEY (store_code) REFERENCES dim_stores_details(store_code);
ALTER TABLE orders_table
ADD CONSTRAINT fk_orders_table_dim_products FOREIGN KEY (product_code) REFERENCES dim_products(product_code);